# Training

In [ ]:
import numpy as np
import cv2
import random
from random import randint
from scipy import ndimage
import os
import sys

from google.colab.patches import cv2_imshow

from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from keras.layers import Dense, Dropout, Flatten, Input, BatchNormalization, Conv2D, MaxPooling2D
from keras.models import Model
from keras import metrics
import keras.backend as backend

from sklearn.metrics import accuracy_score

from tqdm import tqdm
import shutil

In [ ]:
# Here are classes named for training
Dict = {0: 'advance_to_contact', 1: 'ambush', 2: 'attack', 3: 'attack_by_fire', 4: 'block', 
        5: 'breach', 6: 'clear', 7: 'contain', 8: 'control', 9: 'counterattack', 10: 'cover', 
        11: 'delay', 12: 'deny', 13: 'destroy', 14: 'disrupt', 15: 'fix', 16: 'guard', 
        17: 'isolate', 18: 'main_attack', 19: 'neutralize', 20: 'occupy', 21: 'penetrate', 
        22: 'retain', 23: 'retire', 24: 'screen', 25: 'secure', 26: 'seize', 27: 'support_by_fire', 
        28: 'suppress', 29: 'turn', 30: 'withdraw'}

In [ ]:
Dict_images = {key: [] for key in Dict.values()}
Dict_rotations = {key: [] for key in Dict.values()}

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def angle_error_metric(y_true, y_pred):
    diff = 180 - abs(abs(backend.argmax(y_true) - backend.argmax(y_pred)) - 180)
    return backend.mean(backend.cast(backend.abs(diff), backend.floatx()))

def angle_error(y_true, y_pred):
    diff = 180 - abs(abs(y_true - y_pred) - 180)
    return np.abs(diff)

def angle_error_metric_sym(y_true, y_pred):
    diff = 180 - abs(abs(backend.argmax(y_true) - backend.argmax(y_pred)) - 180)
    if diff > 90:
        diff = 180 - diff
    return backend.mean(backend.cast(backend.abs(diff), backend.floatx()))

def angle_error_sym(y_true, y_pred):
    diff = 180 - abs(abs(y_true - y_pred) - 180)
    if diff > 90:
        diff = 180 - diff
    return np.abs(diff)

In [ ]:
def CNNModel():
    classes = 360

    input = Input(shape=(80, 80, 1))
    x = Conv2D(32, kernel_size = (3, 3), strides = (1, 1), padding = 'same', activation='relu')(input)
    x = MaxPooling2D(pool_size=(2, 2), padding = 'same')(x)
    x = BatchNormalization()(x)
    x = Conv2D(32, kernel_size = (3, 3), strides = (1, 1), padding = 'same', activation='relu')(x)
    x = MaxPooling2D(pool_size=(2, 2), padding = 'same')(x)
    x = BatchNormalization()(x)
    x = Conv2D(64, kernel_size = (3, 3), strides = (1, 1), padding = 'same', activation='relu')(x)
    x = MaxPooling2D(pool_size=(2, 2), padding = 'same')(x)
    x = BatchNormalization()(x)
    x = Conv2D(64, kernel_size = (3, 3), strides = (1, 1), padding = 'same', activation='relu')(x)
    x = MaxPooling2D(pool_size=(2, 2), padding = 'same')(x)
    x = BatchNormalization()(x)
    x = Dense(64, activation='relu')(x)
    x = Dense(64, activation='relu')(x)
    x = Flatten()(x)
    x = Dense(classes, activation='softmax')(x)

    model = Model(inputs=input, outputs=x)
    # print(model.summary())

    return model

In [ ]:
!mkdir /content/images
!mkdir /content/images/train
!mkdir /content/labels
!mkdir /content/labels/train
!mkdir /content/rotations
!mkdir /content/rotations/train

In [ ]:
!unzip -q '/content/drive/My Drive/NATO/train.zip' -d '/content/images/train'
!unzip -q '/content/drive/My Drive/NATO/train_l.zip' -d '/content/labels/train'
!unzip -q '/content/drive/My Drive/NATO/train_r.zip' -d '/content/rotations/train'

In [ ]:
images = '/content/images/train'
labels = '/content/labels/train'
rotations = '/content/rotations/train'

file_list = os.listdir(images)

for name in tqdm(file_list):
    tmp_list_class = []
    image = cv2.imread(images+"/"+name, cv2.IMREAD_GRAYSCALE)
    with open(labels+"/"+str(name).split(".")[0]+".txt") as f:
        lines = [line.rstrip('\n') for line in f]
        for line in lines:
            x = int((float(line.split(" ")[1])-(float(line.split(" ")[3])/2)) * 770)
            y = int((float(line.split(" ")[2])-(float(line.split(" ")[4])/2)) * 576)
            w = int(float(line.split(" ")[3]) * 770 + 5)
            h = int(float(line.split(" ")[4]) * 576 + 5)

            crop_image = image[y:y+h, x:x+w]
            tmp_list_class.append(int(line.split(" ")[0]))
            dim = (80, 80)
            resized_img = cv2.resize(crop_image, dim, interpolation = cv2.INTER_AREA)
            Dict_images[Dict[int(line.split(" ")[0])]].append(resized_img)
    with open(rotations+"/"+str(name).split(".")[0]+".txt") as a:
        r = [line.rstrip('\n') for line in a]
        i = 0
        for rotation in r:
            Dict_rotations[Dict[tmp_list_class[i]]].append(rotation)
            i += 1

100%|██████████| 150000/150000 [07:16<00:00, 343.62it/s]


In [ ]:
for i in Dict_images.keys():
    print(str(i))
    cv2_imshow(Dict_images[str(i)][0])

In [ ]:
for i in Dict_images.keys():
    print(str(i))
    x = Dict_images[str(i)]
    y = Dict_rotations[str(i)]

    y_categorical = to_categorical(y, 360)

    x_train = np.array(x[0:])
    y_train = np.array(y_categorical)

    x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], 1)

    print(x_train.shape, y_train.shape)

    model = CNNModel()

    # Compile the model
    model.compile(loss='categorical_crossentropy',
                optimizer='adam',
                metrics=[metrics.categorical_accuracy, angle_error_metric])

    # Fit the model
    model.fit(x_train, y_train, batch_size = 64, epochs = 200)

    model.save(str(i)+"_rotation_model_31_1203.h5")

    colab_link = '/content/'+str(i)+'_rotation_model_31_1203.h5'
    gdrive_link = "/content/drive/My Drive/models_rotation/1203/"
    shutil.copy(colab_link, gdrive_link)

In [ ]:
for i in Dict_images.keys():
    colab_link = '/content/'+str(i)+'_rotation_model_31_1003.h5'
    gdrive_link = "/content/drive/My Drive/models_rotation/"
    shutil.copy(colab_link, gdrive_link)

# Prediction

In [ ]:
!mkdir /content/images
!mkdir /content/images/test
!mkdir /content/labels
!mkdir /content/labels/test
!mkdir /content/rotations
!mkdir /content/rotations/test

In [ ]:
!unzip -q '/content/drive/My Drive/NATO/test.zip' -d '/content/images/test'
!unzip -q '/content/drive/My Drive/NATO/test_l.zip' -d '/content/labels/test'
!unzip -q '/content/drive/My Drive/NATO/test_r.zip' -d '/content/rotations/test'

In [ ]:
Dict_images_test = {key: [] for key in Dict.values()}
Dict_rotations_test = {key: [] for key in Dict.values()}
Dict_predicted_test = {key: [] for key in Dict.values()}
Dict_angle_errors_test  = {key: [] for key in Dict.values()}

In [ ]:
images_test = '/content/images/test'
labels_test = '/content/labels/test'
rotations_test = '/content/rotations/test'

file_list = os.listdir(images_test)
model = CNNModel()

for name in tqdm(file_list):
    tmp_list_class = []
    image = cv2.imread(images_test+"/"+name, cv2.IMREAD_GRAYSCALE)
    with open(labels_test+"/"+str(name).split(".")[0]+".txt") as f:
        lines = [line.rstrip('\n') for line in f]
        for line in lines:
            x = int((float(line.split(" ")[1])-(float(line.split(" ")[3])/2)) * 770)
            y = int((float(line.split(" ")[2])-(float(line.split(" ")[4])/2)) * 576)
            w = int(float(line.split(" ")[3]) * 770 + 5)
            h = int(float(line.split(" ")[4]) * 576 + 5)

            crop_image = image[y:y+h, x:x+w]
            tmp_list_class.append(int(line.split(" ")[0]))
            dim = (80, 80)
            resized_img = cv2.resize(crop_image, dim, interpolation = cv2.INTER_AREA)
            Dict_images_test[Dict[int(line.split(" ")[0])]].append(resized_img)

            ## Predict
            reshaped_img = resized_img.reshape(1, 80, 80)
            try:
                model.load_weights('/content/drive/MyDrive/models_rotation/1203/'+Dict[int(line.split(" ")[0])]+'_rotation_model_31_1203.h5')
                predicted_rotation = model.predict(reshaped_img, verbose=0)
                Dict_predicted_test[Dict[int(line.split(" ")[0])]].append(np.argmax(predicted_rotation))
            except OSError:
                continue
    with open(rotations_test+"/"+str(name).split(".")[0]+".txt") as a:
        r = [line.rstrip('\n') for line in a]
        i = 0
        for rotation in r:
            Dict_rotations_test[Dict[tmp_list_class[i]]].append(rotation)
            i += 1
    

100%|██████████| 7000/7000 [53:57<00:00,  2.16it/s]


In [ ]:
for i in Dict_images.keys():
    print(str(i))
    symbol_name = str(i)

    print(len(Dict_rotations_test[symbol_name]))
    try:
        if str(i) in ['destroy', 'neutralize', 'suppress']:
            for i in range(len(Dict_rotations_test[symbol_name])):
                Dict_angle_errors_test[symbol_name].append(angle_error_sym(int(Dict_rotations_test[symbol_name][i]), int(Dict_predicted_test[symbol_name][i])))
        else:
            for i in range(len(Dict_rotations_test[symbol_name])):
                Dict_angle_errors_test[symbol_name].append(angle_error(int(Dict_rotations_test[symbol_name][i]), int(Dict_predicted_test[symbol_name][i])))

        print(len(Dict_angle_errors_test[symbol_name]), round(np.mean(Dict_angle_errors_test[symbol_name]), 2))

        kokku = sum(i <= 30 for i in Dict_angle_errors_test[symbol_name])
        print(kokku, str(round(kokku/len(Dict_angle_errors_test[symbol_name])*100, 2))+"%")
    except IndexError:
        continue

advance_to_contact
995
995 11.42
957 96.18%
ambush
1053
1053 10.38
1025 97.34%
attack
987
987 9.33
954 96.66%
attack_by_fire
1042
1042 9.73
1022 98.08%
block
1048
1048 11.48
1010 96.37%
breach
990
990 8.87
977 98.69%
clear
1099
1099 10.34
1073 97.63%
contain
1027
1027 10.08
1008 98.15%
control
1043
1043 16.68
971 93.1%
counterattack
979
979 11.47
915 93.46%
cover
986
986 13.55
937 95.03%
delay
1007
1007 10.07
987 98.01%
deny
983
983 12.61
946 96.24%
destroy
1036
1036 11.89
981 94.69%
disrupt
983
983 11.67
948 96.44%
fix
1000
1000 11.08
965 96.5%
guard
1016
1016 12.0
980 96.46%
isolate
979
979 15.85
920 93.97%
main_attack
980
980 18.22
857 87.45%
neutralize
1010
1010 17.69
873 86.44%
occupy
958
958 13.21
914 95.41%
penetrate
1061
1061 9.22
1040 98.02%
retain
994
994 14.78
940 94.57%
retire
1008
1008 9.25
993 98.51%
screen
1018
1018 13.92
965 94.79%
secure
975
975 11.6
937 96.1%
seize
1075
1075 21.33
831 77.3%
support_by_fire
1016
1016 9.03
1000 98.43%
suppress
1044
1044 21.25
852 81.61%

In [ ]:
symbol_name = "advance_to_contact"

res = tuple(zip(Dict_angle_errors_test[symbol_name], Dict_rotations_test[symbol_name], Dict_predicted_test[symbol_name]))
sorted(res, key = lambda x: (-x[0],x[1],x[2]))